#Tutorial Caffe + Digits 

##TODO: Preparar título, nombres, logos...etc



#Deep Learning with Caffe

Framework desarrollado en la Universidad de Berkeley usando C++ y CUDA, cuenta con interfaces de uso en Python, Matlab y Linea de Comandos. 


Caffe permite crear una arquitectura de red de manera sencilla, definir hiperparámetros para el entrenamiento y cuenta con una gran cantidad de capas predefinidas, modelos entrenados para realizar clasificación de imagenes.

Caffe: https://github.com/BVLC/caffe

<iframe src="http://demo.caffe.berkeleyvision.org/" width=800 height=800></iframe>

##Clasificación de Imágenes con Caffe



<img src="imagenes/mylenet.png">



_Instruccions detalladas para el entrenamiento en Caffe en el siguiente [enlace](http://nbviewer.ipython.org/urls/bitbucket.org/sandiego206/datasets-training/raw/d0c609170f5afaf1881ced95363a21b74dbf5367/Instrucciones.ipynb)_

##1.1 Preprocesamiento de Imágenes

* Es recomendable tener una gran cantidad de datos en el dataset, en este punto es posible aumentarlos realizando un preprocesamiento en la capa de datos en Caffe, por defecto este incluye:

    - Mirroring
    - Random crop
    - Mean substraction
    - Scaling

##1.2 Balanceo de Clases

* Para evitar resultados falsamente optimistas es recomendable tener un balanceo de clases que permita a la red neuronal aprender de todos los datos de entrenamiento.

<img src="imagenes/caltech101.png" width=600 height=300>

##1.3 Preparar el dataset

La primera capa de la red neuronal en Caffe, permite leer los datos en diferentes formatos.

* __Image_Data__: Requiere un csv con el path de cada imagen y la clasificación de estas.<br>
<br>

* __LMDB__:&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Recomendado para datasets de __gran__ tamaño otorga mejor rendimiento I/O, los mismos requisitos que Image_Data.<br>
<br>

* __Window__: &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Lee ventanas en la imagen, usado para detección de objetos.

##1.4 Data Layer

---

``` python
    name: "CaltechNet"
    layer {
      name: "data" 
      type: DATA
      top: "data"
      top: "label"
      data_param {
        source: "/bigdata/drueda/datasets_lmdb/caltech_train_lmdb"
        backend: LMDB
        batch_size: 16
      }
      transform_param {
        crop_size: 227
        mean_file: "/bigdata/drueda/datasets_lmdb/caltech_mean.binaryproto"
        mirror: True
        }
      include: { phase: TRAIN }
    }
```

---

##2.1 Convolutional Layer
---
<img src="imagenes/convolution.gif" width=400 height=400 align=left>


<img src="imagenes/convolution_text.png" align=center>


##2.2 Activation Layer

<img src="imagenes/relu.png"  align=left>
<img src="imagenes/relu_text.png" align=center>

##2.3 Pooling Layer
---

<img src="imagenes/pooling.gif" width=50% align=left >
<img src="imagenes/pooling_text.png" align=center >

##2.4 Fully Connected Layer

<img src="imagenes/fully_connected.gif" align=left >

<img src="imagenes/fully_connected_text.png" align=center >

##2.5 Solver Prototxt

En este archivo definimos los parámetros globales que rigen el entrenamiento de la red neuronal.

``` python
test_iter: 8 # Especifica cuantos pases adelante la prueba hará
test_interval: 29 # Iteraciones para una validación del entrenamiento
base_lr: 0.001 # Tasa de aprendizaje
display: 3 # Iteraciones para mostrar información de la función de costo (loss)
max_iter: 2900 # Número máximo de iteraciones
lr_policy: "step" # Política de disminución de la tasa de aprendizaje
gamma: 0.1 # Valor en que disminuimos base_lr

# Momentum y Weight decay de la red neuronal
momentum: 0.9 
weight_decay: 0.0005

stepsize: 957 # Número de iteraciones necesarias para disminuir la tasa de aprendizaje.
snapshot: 29 # Número de iteraciones en que tomaremos una "foto" del estado de la red.
snapshot_prefix: "snapshot" # Sufijo de la "foto" a la red neuronal

#Modo de ejecución CPU/GPU
solver_mode: GPU

# Nombre de la red que entrenaremos en formato prototxt
net: "train_val.prototxt"
# Modo de evaluación (Stochastic Gradient Descent)
solver_type: SGD

#Opcional, semilla para iniciar los randómicos
random_seed: 64

```


##Iniciar un entrenamiento con Caffe

``` bash

./{CAFFE_PATH}/build/tools/caffe train --solver=solver.prototxt

```

##Reanudar un entrenamiento

``` bash

./{CAFFE_PATH}/build/tools/caffe train --solver=solver.prototxt \
    --snapshot=snapshot_train_iter_10000.solverstate

```

#Finetuning

``` bash

./{CAFFE_PATH}/build/tools/caffe train --solver solver.prototxt \
    --weights bvlc_reference_caffenet.caffemodel

```

#Digits

Digits: https://github.com/NVIDIA/DIGITS/blob/master/docs/InstallCaffe.md